In [120]:
import requests
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import os
from datetime import datetime as dt
import os.path
from os import path
import sys
from importlib import reload
import copy
import pickle

In [15]:
sys.path.append('..') #agregar directorio madre del repo para poder importar librerias customizadas
#importar librerias customizadas

In [199]:
import modules.Elastic_Module
reload(modules.Elastic_Module)

<module 'modules.Elastic_Module' from '../modules/Elastic_Module.py'>

In [200]:
from modules.Elastic_Module import *

In [2]:
ls

 CKAN_to_elastic_encrypted.ipynb  'kibana_migrate_index (1) (1).ipynb'


In [3]:
url_ckan='http://datos.gob/api/3/action/package_list'
url_package='http://datos.gob/api/3/action/package_show?id='
url_package_search='http://datos.gob/api/3/action/package_search?'

In [4]:
#Listas de paquetes ckan (con sus ID)
ckan_packages=requests.get(url_ckan) #consultar api
ckan_packages=ckan_packages.json() #convertir respuesta a json

In [5]:
ckan_packages

{'help': 'http://datos.gob/api/3/action/help_show?name=package_list',
 'success': True,
 'result': ['10000',
  '10002',
  '10006',
  '10007',
  '10010',
  '10011',
  '10014',
  '10029',
  '10037',
  '10043',
  '10049',
  '10054',
  '10063',
  '10069',
  '10071',
  '10078',
  '10080',
  '10101',
  '10103',
  '10113',
  '10126',
  '10127',
  '10138',
  '10145',
  '10146',
  '10154',
  '10201',
  '10203',
  '10206',
  '10208',
  '10223',
  '10232',
  '10272',
  '10279',
  '10284',
  '10286',
  '10288',
  '1030',
  '1036',
  '10385',
  '1044',
  '10451',
  '10453',
  '1047',
  '1048',
  '1049',
  '1050',
  '1051',
  '1052',
  '1053',
  '1054',
  '1055',
  '1056',
  '1057',
  '1058',
  '1059',
  '10616',
  '1063',
  '1065',
  '10657',
  '10661',
  '10662',
  '10664',
  '10667',
  '10670',
  '10671',
  '10674',
  '10675',
  '10678',
  '10687',
  '10713',
  '1073',
  '10740',
  '1080',
  '10807',
  '10808',
  '10809',
  '1081',
  '1082',
  '10825',
  '10826',
  '10827',
  '10828',
  '10829',


In [6]:
#OBTENER TODOS LOS PAQUETES DE LA BASE DE DATOS CKAN
rows=str(1000) #maximo numero de resultados por consulta (maximo permitido es 1000)
packs=[] #contenedor de paquetes retornados
for s in range(0,4):
    start=str(s*int(rows)) #paginacion
    packages=requests.get(url_package_search+'rows='+rows+'&'+'start='+start)
    packages=packages.json()
    packs.extend(packages['result']['results'])

In [75]:
#revisar root de los paquetes
#root={}
#for k,v in packs[0].items():
#    if (k!='resources') & (k!='organization') :
#        root[k]=v

In [114]:
recursos=[]
for p in packs:
    for r in p['resources']:
        d=copy.deepcopy(r)
        #parámetros del paquete
        d['package_id']=p['id'] #id
        d['package_name']=p['name'] #nombre
        d['package_title']=p['title'] #titulo
        d['package_notes']=p['notes'] #notas
        if 'organization' in p.keys(): #si es que existe info de organizacion
            if p['organization']!=None:
                d['inst.']=p['organization']['title'] #si es que existe info del titulo incluirlo
                d['inst_name']=p['organization']['name'] #si es que existe info del titulo incluirlo
                d['inst_id']=p['organization']['id'] #si es que existe info del id inst.
            else:
                d['inst.']=None
                d['inst_name']=None
                d['inst_id']=None
        recursos.append(d)
recursos=pd.DataFrame(recursos) #convertir a df        

In [115]:
recursos=recursos.rename(columns={'last_modified':'fecha','format':'formato'}) #cambiar nombres de columnas

In [116]:
recursos=recursos.drop(['mimetype','cache_url','url','cache_last_updated','hash','url_type','webstore_url','webstore_last_updated',
              'mimetype_inner','position','resource_type'],axis=1) #botar columnas

In [117]:
recursos['fecha']=pd.to_datetime(recursos['fecha']) #convertir la fecha a formato solo con la fecha (sin la hora)
#desagregar en dia, mes, año
recursos['dia']=recursos['fecha'].dt.date
recursos['year']=recursos['fecha'].dt.year
recursos['mes']=recursos['fecha'].dt.month

In [122]:
#guardar paso en csv
path='/data/cache/ckan/state_datos_gob'
now = dt.now()
date_time = now.strftime("_%m-%d-%Y_%H-%M-%S")

In [123]:
recursos.to_csv(path+date_time+'.csv')

In [124]:
#guardar data de paquetes cruda en pickle y borrar para liberar espacio en memoria
with open(path+date_time+'.pickle', 'wb') as f:
    pickle.dump(packs, f)
del(packs)

In [82]:
#RUTA PARA RECUPERAR PASO GUARDADO EN CSV
#ls /data/cache/ckan/state_datos_gob*

In [83]:
#RECUPERAR PASO GUARDADO EN CSV
#recursos=pd.read_csv('/data/cache/ckan/state_datos_gob_03-31-2020_05-05-15.csv',index_col=0)

In [84]:
data=recursos.to_dict(orient='records')

In [91]:
del(data)

In [87]:
#variables de entorno
args={}
args['elastic_user']='elastic'
args['elastic_psswd']=os.environ['ELASTIC_PASSWD']

In [88]:
ES_HOST = "https://"
basic_auth={'user':args['elastic_user'], 'psswd':args['elastic_psswd']}

In [201]:
es=elastic_connection(ES_HOST,basic_auth=basic_auth)

In [202]:
es.ping()

True


In [204]:
es.generateID_params_custom=generateID_params_custom

In [205]:
es.add_docs(recursos)

In [206]:
indexes=[
    ('ckan','ckan')
        ]

In [207]:
#generar bulk data
i=indexes[0]
n=1500
id_field='id'
es.bulk_data(INDEX_NAME=i[0],_type=i[1],n=n,id_field=id_field)
#test

creando paquetes de data bulk
size of bulk docs is : 0.1953125 Kbs
N° docs: 10132.0, N° chunks: 14


In [208]:
len(es.chunks[0])

1500

In [209]:
#generateID_params={'x':'_id','y':'e_id'} #cambiar nombres de campos (parámetros)

In [210]:
#generar campos concatenados (parámetros)
generateID_params_custom={'fields':['package_id','id'], #campos a concatenar
                               'head':20, #numero de caracteres a dejar
                                'sep':'_' #separador entre campos
                               }

In [211]:
#prepar la data para elastic search
es.elastic_prepare_pattern()

In [ ]:
#es.chunks[0]

In [213]:
#esear_index(i[0])

In [214]:
es.bulk_index()

bulk indexing...:index---> ckan
 indexing chunk : 13 of 14finished bulk indexing


ValueError: No objects to concatenate

In [215]:
es.test_index(i[0],n=15)

 response: '{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'ckan', '_type': 'ckan', '_id': '0575f658-4529-4659-bc44-649f51ebd2f5_750', '_score': 1.0, '_source': {'description': 'Información de Ejecución Presupuestaria a Enero de 2019 del Gobierno Central a nivel Programa. ', 'name': 'A nivel Programa', 'formato': 'CSV', 'package_id': 'da63c256-d0f2-4f6b-b2ea-4332f4fe6f7e', 'created': '2019-03-04T15:16:53.778614', 'state': 'active', 'fecha': '2019-06-10T21:51:02.364036', 'revision_id': 'bb261f01-23db-418c-acf4-194dc3ed7b57', 'size': None, 'package_name': 'ejecucion-presupuestaria-a-enero-de-2019-del-gobierno-central', 'package_title': 'Ejecución Presupuestaria a Enero de 2019 del Gobierno Central', 'package_notes': 'Información de Ejecución Presupuestaria a Enero de 2019 del Gobierno Central, presentada en reportes a nivel Nacional, Partida, 

In [216]:
es.count_documents(i[0])

[{'epoch': '1586336107', 'timestamp': '08:55:07', 'count': '10057'}]
